# 0.0 Importando Biblioteca

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime

# Funções

In [32]:
def get_data(path):
    data = pd.read_csv(path)
    
    return data

pd.set_option('display.float_format', lambda x : '%.2f' % x)

# 1.0 Data

# Extraction

# 1.1 Load data

In [33]:
df = get_data('kc_house_data.csv')

# 1.2 Dataset Dimension

In [34]:
print('Number of columns: {}'.format(df.shape[1]))
print('Number of rows: {}'.format(df.shape[0]))

Number of columns: 21
Number of rows: 21613


# 1.3 Checking datatypes

In [35]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

# Transformation

In [36]:
# Convert column date from object type to datetime type
df['date'] = pd.to_datetime(df['date'])

# 1.4 Cleaning data

# 1.4.1 Check null values

In [37]:
df.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

# Clean Data

# 1.4.2 Checking Duplicates

In [38]:
df.duplicated().sum()

0

# 1.4.3 Drop duplicate columns

In [39]:
df = df.drop(columns = ['sqft_lot15', 'sqft_living15'])

# 1.5 Descriptive Stats

In [40]:
# select only data types equals to int64 and float64
num_attri = df.select_dtypes(include = ['int64', 'float64'])

# exclude the id column
num_attri = num_attri.iloc[:, 1:]

mean = pd.DataFrame(num_attri.apply(np.mean))
median = pd.DataFrame(num_attri.apply(np.median))
std = pd.DataFrame(num_attri.apply(np.std))
max_ = pd.DataFrame(num_attri.apply(np.max))
min_ = pd.DataFrame(num_attri.apply(np.min))

# concatenate all created variables
df0 = pd.concat([max_, min_, mean, median, std ], axis = 1).reset_index()

# rename columns
df0.columns = ['attributes', 'max', 'min', 'mean', 'median', 'std']

In [41]:
df0

,attributes,max,min,mean,median,std
0,price,7700000.00,75000.00,540088.14,450000.00,367118.70
1,bedrooms,33.00,0.00,3.37,3.00,0.93
2,bathrooms,8.00,0.00,2.11,2.25,0.77
3,sqft_living,13540.00,290.00,2079.90,1910.00,918.42
4,sqft_lot,1651359.00,520.00,15106.97,7618.00,41419.55
5,floors,3.50,1.00,1.49,1.50,0.54
6,waterfront,1.00,0.00,0.01,0.00,0.09
7,view,4.00,0.00,0.23,0.00,0.77
8,condition,5.00,1.00,3.41,3.00,0.65
9,grade,13.00,1.00,7.66,7.00,1.18


# 1.6 New Features

In [42]:
data = pd.DataFrame()

data = df[['zipcode', 'price']].groupby('zipcode').median().reset_index()

df2 = pd.merge(df, data, on = 'zipcode', how = 'inner')

df2 = df2.rename(columns = {'price_x' : 'price', 'price_y' : 'price_median' })

In [117]:
# feature for hypothesis 1
df['waterview'] = df['waterfront'].apply(lambda x : 'no' if (x == 0) else 'yes')

In [75]:
# feature for hypothesis 2
df['yr_b_mean'] = df['yr_built'].apply(lambda x : '< 1955' if (x < 1955) else '> 1955' )

In [115]:
# feature for hypothesis 3
df['has_basement'] = df['sqft_basement'].apply(lambda x : 'no' if (x == 0) else 'yes')

In [90]:
# feature for hypothesis 4
df['year'] = pd.to_datetime(df['date']).dt.year

In [105]:
# feature for hypothesis 5
df['month'] = pd.to_datetime(df['date']).dt.month

# 2.0 Answering hypothesis

# Hypothesis 1: Usually real state with waterfront view are more 30% expensive in average

In [119]:
h1 = df[['waterview', 'price']].groupby('waterview').mean().reset_index()

fig = px.bar(h1, x = 'waterview', y = 'price', title = 'Waterfront View Average Price', height = 700 )
fig.show()

# Hypothesis 2: Real state with year of construction less than 1955 are 50% cheaper in average

In [81]:
h2 = df[['yr_b_mean', 'price']].groupby('yr_b_mean').mean().reset_index()

fig = px.bar(h2, x = 'yr_b_mean', y = 'price', title = 'Construction Year Average Price', height = 700 )
fig.show()

# Hypothesis 3: Real state without basement, have a greater sqft lot about 40% in average

In [116]:
h3 = df[['has_basement', 'sqft_lot']].groupby('has_basement').sum().reset_index()

fig = px.bar(h3, x = 'has_basement', y = 'sqft_lot', title = 'Basement x No Basement Average Price', height = 700 )
fig.show()

# Hypothesis 4: The price growth YoYof real state is 10%

In [101]:
h4 = df[['year', 'price']].groupby('year').sum().reset_index()

fig = px.bar(h4, x = 'year', y = 'price', title = 'Growth YoY Average Price', height = 700 )
fig.show()

# Hypothesis 5: Real state with 3 bathrooms have a price growth MoM of 15% 

In [112]:
h5 = df[['bathrooms','month', 'price']].groupby(['bathrooms', 'month']).sum().reset_index()
h5 = h5[h5['bathrooms']==3]

fig = px.line(h5, x = 'month', y = 'price', title = 'Real state 3 Bathrooms Price Growth', height = 700 )
fig.show()

# Hypothesis

# Hypothesis

# Hypothesis

# Hypothesis

# Hypothesis

In [114]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'yr_b_mean', 'has_basement', 'year', 'month'],
      dtype='object')

# 3.0 Business Questions